In [188]:
from database_cloudant import CloudantDB
from cloudant.design_document import DesignDocument
from cloudant.view import View
import sys



#db1 = CloudantDB("tweets_keywords")    # tweets in the specified location with keywords 
#db2 = CloudantDB("tweets_no_keywords") # tweets in the sepcified locations without keyowrds
#db3 = CloudantDB("tweets_raw")         # raw tweet data in the specified locations
#db4=CloddantDB("")
#print(len(db1.get_data()))
#print(db2.get_data(n=10000))
#print(len(db3.get_data(n=10)))



    
def dataofday(date):    
    # get all data from a specific day
    db1 = CloudantDB("tweets_keywords")    # tweets in the specified location with keywords 
    db2 = CloudantDB("tweets_no_keywords") # tweets in the sepcified locations without keyowrds
    n = 100  # number of tweets need to retrieve
    ddoc = DesignDocument(db2.curDB, '_design/ddoc001')
    ddoc.fetch()
    view = View(ddoc,"view_time")
    data = []
    for row in view(group=False,limit=n,reduce=False,key=date,include_docs=True)["rows"]:
        data.append(row)
    
    db1.close()
    db2.close()
    return data

    
    

    
#db3.close()
def city_daily_popular_10words(tweetlist):
    city=[]
    for k in tweetlist:
        if k["zone"] not in city:
            city.append(k["zone"])
    alltext={}
    for c in city:
        alltext[c]=[]
    for k in tweetlist:
        for c in city:
            if k["zone"] in c:
                alltext[c].append(k["text"])
    popdict={}             
    for c in city:
        popdict[c]=top_words(alltext[c])
    return popdict


if __name__ == "main":
    ipaddress=sys.argv[-1]

    date = "2021-01-01"
    data=dataofday(date)

    citytopwords=city_daily_popular_10words(data)
    print(citytopwords)


In [189]:
def get_data_summary(db="covid",viewType="month",startkey="2020-10",mode = "mean"):
    if(db=="covid"):
        db = CloudantDB('tweets_covid')
    elif(db=="vaccine"):
        db= CloudantDB("tweets_vaccine")
    elif(db=="no_keywords"):
        db=CloudantDB('tweets_no_keywords')
    else:
        print("db parameter must in [covid,vaccine,no_keywords]")
        return {}
    ddoc = DesignDocument(db.curDB,'_design/ddoc001')
    ddoc.fetch()
    if(viewType=="month"):
        view = View(ddoc, 'view_month')
    elif(viewType=="zone"):
        view = View(ddoc,"view_zone")
    elif(viewType=="zone month"):
        view=View(ddoc, 'view_zone_month')
    else:
        view = View(ddoc, 'view_time')
    result = {}
    for row in view(limit=1000000,reduce=True,group=True,startkey=startkey)['rows']:
        if(mode=="mean"):
            result[row['key']] = round(row['value']['sum']/row['value']['count'],6)
        elif(mode=="count"):
            result[row['key']] = row['value']['count']
    return result

def get_data(n=100,db="covid",viewType="day",startkey="2021-05"):
    if(db=="covid"):
        db = CloudantDB('tweets_covid')
    elif(db=="vaccine"):
        db= CloudantDB("tweets_vaccine")
    elif(db=="no_keywords"):
        db=CloudantDB('tweets_no_keywords')
    
    else:
        print("db parameter must in [covid,vaccine,no_keywords]")
        return []
    ddoc = DesignDocument(db.curDB,'_design/ddoc001')
    ddoc.fetch()
    if(viewType=="zone"):
        view = View(ddoc,"view_zone")
    elif(viewType=="day"):
        view = View(ddoc, 'view_time')
    elif(viewType=="month"):
        view = View(ddoc, 'view_month')
    else:
        return "Does not support viewType"+viewType
    result = []
    for row in view(limit=n,reduce=False,group=False,startkey=startkey,include_docs=True)['rows']:
        result.append(row)
    return result


In [190]:
get_data(db="covid",viewType="month")

ConnectionError: HTTPConnectionPool(host='172.26.131.97', port=5984): Max retries exceeded with url: /_session (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001CEF2829A90>: Failed to establish a new connection: [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。'))

In [67]:
monthsentidict=get_data_summary(viewType="zone")

Username: admin
Databases: ['test', 'topwords', 'tweets_covid', 'tweets_keywords', 'tweets_no_keywords', 'tweets_raw', 'tweets_vaccine']
Accessing db: tweets_covid


In [68]:
get_data_summary(viewType="zone month")
get_data_summary(db="vaccine",viewType="zone month")
get_data_summary(db="vaccine",viewType="zone")
get_data_summary(viewType="zone")

Username: admin
Databases: ['test', 'topwords', 'tweets_covid', 'tweets_keywords', 'tweets_no_keywords', 'tweets_raw', 'tweets_vaccine']
Accessing db: tweets_covid
Username: admin
Databases: ['test', 'topwords', 'tweets_covid', 'tweets_keywords', 'tweets_no_keywords', 'tweets_raw', 'tweets_vaccine']
Accessing db: tweets_vaccine
Username: admin
Databases: ['test', 'topwords', 'tweets_covid', 'tweets_keywords', 'tweets_no_keywords', 'tweets_raw', 'tweets_vaccine']
Accessing db: tweets_vaccine
Username: admin
Databases: ['test', 'topwords', 'tweets_covid', 'tweets_keywords', 'tweets_no_keywords', 'tweets_raw', 'tweets_vaccine']
Accessing db: tweets_covid


{'Adelaide': 0.596067,
 'Ballarat': 0.737719,
 'Brisbane': 0.567257,
 'Bunbury': 0.688183,
 'Canberra': 0.518975,
 'Geelong': 0.611571,
 'Hobart': 0.48139,
 'Melbourne': 0.589867,
 'Newcastle': 0.714606,
 'Perth': 0.56422,
 'Sydney': 0.542216}

In [69]:
monthsentidict

{'Adelaide': 0.596067,
 'Ballarat': 0.737719,
 'Brisbane': 0.567257,
 'Bunbury': 0.688183,
 'Canberra': 0.518975,
 'Geelong': 0.611571,
 'Hobart': 0.48139,
 'Melbourne': 0.589867,
 'Newcastle': 0.714606,
 'Perth': 0.56422,
 'Sydney': 0.542216}

In [70]:
get_data(db="no_keywords",viewType="month")

Username: admin
Databases: ['test', 'topwords', 'tweets_covid', 'tweets_keywords', 'tweets_no_keywords', 'tweets_raw', 'tweets_vaccine']
Accessing db: tweets_no_keywords


[{'id': '1388282235938394130',
  'key': '2021-05',
  'value': 0.98362815,
  'doc': {'_id': '1388282235938394130',
   '_rev': '1-bdd52374143a46cb27c3e48f87628d8e',
   'id': 1388282235938394130,
   'user_id': 2225880756,
   'created_at': '2021-05-01 00:00:52',
   'text': 'Just posted a photo @ Bondi Beach https://t.co/wL81mg2GsI',
   'place': {'type': 'city',
    'name': 'Sydney',
    'fullname': 'Sydney, New South Wales',
    'bounding_box': '[[[150.520928608, -34.1183470085], [151.343020992, -34.1183470085], [151.343020992, -33.578140996], [150.520928608, -33.578140996]]]'},
   'sentiment_score': '0.98362815',
   'sentiment': 'Positive',
   'zone': 'Sydney',
   'keywords': 'no keywords'}},
 {'id': '1388282374153117700',
  'key': '2021-05',
  'value': 0.76601136,
  'doc': {'_id': '1388282374153117700',
   '_rev': '1-02412e724ab78d53e8263167a034226e',
   'id': 1388282374153117700,
   'user_id': 835298557,
   'created_at': '2021-05-01 00:01:25',
   'text': 'Professor Clifton provides a ve

In [146]:
from collections import Counter

In [147]:
wordfreqdict=Counter([1,2,3,4,5,6,2,3,4,5])

In [149]:
wordfreqdict.most_common(3)

[(2, 2), (3, 2), (4, 2)]

In [181]:
get_data(n=70,db="no_keywords",viewType="month")

ConnectionError: HTTPConnectionPool(host='172.26.131.97', port=5984): Max retries exceeded with url: /_session (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001CE808ADD30>: Failed to establish a new connection: [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。'))

In [170]:
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords


def top_words(textlist,top=10):
    stop_words = stopwords.words('english')
    wordfreqdict=Counter()
    popular=[]
    tokenizer=TweetTokenizer()
    
    for t in textlist:
        token = tokenizer.tokenize(t)
        for w in token:
            if len(w)>=5 and w not in stop_words:
                wordfreqdict[w]+=1

    return wordfreqdict.most_common(top)

def city_daily_popular_10words(viewType="month",startkey="2021-05"):

    city=[]
    tweetlist=get_data(n=70000,db="no_keywords",viewType="month", startkey=startkey)
    for k in  tweetlist:
        if k["doc"]["zone"] not in city:
            city.append(k["doc"]["zone"])
    alltext={}
    for c in city:
        alltext[c]=[]
    for k in tweetlist:
        for c in city:
            if k["doc"]["zone"] in c:
                alltext[c].append(k["doc"]["text"])
    popdict={}             
    for c in city:
        popdict[c]=dict(top_words(alltext[c]))
    return popdict

In [177]:
db10 = CloudantDB("topwordsmonth5")
for i in range(1,10):
    startkey="2020-0"+str(i)
    
    cdict=city_daily_popular_10words(startkey)
    #print(cdict)
    db10.add_record(cdict,key=startkey)
    
    
print(db10.curDB["2020-01"])
print(db10.curDB["2020-02"])
print(db10.curDB["2020-03"])

Username: admin
Databases: ['topwordsmonth3', 'topwordsmonth5', 'topwordsmonth6', 'topwordsmonth7', 'topwordstotal', 'tweets_covid', 'tweets_keywords', 'tweets_no_keywords', 'tweets_raw', 'tweets_vaccine']
Accessing db: topwordsmonth5
Username: admin
Databases: ['topwordsmonth3', 'topwordsmonth5', 'topwordsmonth6', 'topwordsmonth7', 'topwordstotal', 'tweets_covid', 'tweets_keywords', 'tweets_no_keywords', 'tweets_raw', 'tweets_vaccine']
Accessing db: tweets_no_keywords
Username: admin
Databases: ['topwordsmonth3', 'topwordsmonth5', 'topwordsmonth6', 'topwordsmonth7', 'topwordstotal', 'tweets_covid', 'tweets_keywords', 'tweets_no_keywords', 'tweets_raw', 'tweets_vaccine']
Accessing db: tweets_no_keywords
Username: admin
Databases: ['topwordsmonth3', 'topwordsmonth5', 'topwordsmonth6', 'topwordsmonth7', 'topwordstotal', 'tweets_covid', 'tweets_keywords', 'tweets_no_keywords', 'tweets_raw', 'tweets_vaccine']
Accessing db: tweets_no_keywords
Username: admin
Databases: ['topwordsmonth3', 't

In [179]:
print(db10.curDB["2020-01"])
print(db10.curDB["2020-02"])
print(db10.curDB["2020-09"])

{'_id': '2020-01', 'Sydney': {'posted': 823, 'photo': 789, 'Sydney': 743, 'Australia': 695, 'today': 243, 'people': 222, 'South': 173, 'would': 169, 'think': 167, 'Wales': 156}, 'Adelaide': {'Adelaide': 107, 'Australia': 70, 'South': 60, 'today': 54, 'people': 53, 'photo': 48, 'posted': 43, 'think': 36, 'would': 33, '@CheapFuelAdel': 33}, 'Melbourne': {'posted': 704, 'photo': 679, 'Melbourne': 409, 'Victoria': 341, 'Australia': 319, 'today': 260, 'people': 248, 'would': 202, 'great': 202, 'think': 187}, 'Newcastle': {'posted': 38, 'photo': 38, 'Newcastle': 31, 'people': 17, 'today': 16, 'great': 13, 'first': 13, 'Kitchen': 12, '#couchpeloton': 12, 'Thanks': 11}, 'Brisbane': {'posted': 238, 'photo': 236, 'Brisbane': 128, 'Australia': 104, 'Queensland': 97, 'today': 89, 'people': 77, 'first': 66, 'would': 64, 'really': 60}, 'Canberra': {'Canberra': 72, 'Australia': 49, 'receiving': 47, 'carrier': 45, 'strength': 45, 'Frequency': 44, 'Signal': 44, 'people': 39, 'TURBO': 38, 'great': 31}, 

In [176]:
db10 = CloudantDB("topwordsmonth7")

startkey="2020-01"

cdict=city_daily_popular_10words(startkey)
#print(cdict)
db10.add_record(cdict,key=startkey)
    
    
print(db10.curDB["2020-01"])

Username: admin
Databases: ['topwordsmonth3', 'topwordsmonth5', 'topwordsmonth6', 'topwordstotal', 'tweets_covid', 'tweets_keywords', 'tweets_no_keywords', 'tweets_raw', 'tweets_vaccine']
creating db: topwordsmonth7
Accessing db: topwordsmonth7
Username: admin
Databases: ['topwordsmonth3', 'topwordsmonth5', 'topwordsmonth6', 'topwordsmonth7', 'topwordstotal', 'tweets_covid', 'tweets_keywords', 'tweets_no_keywords', 'tweets_raw', 'tweets_vaccine']
Accessing db: tweets_no_keywords
{'_id': '2020-01', 'Sydney': {'posted': 823, 'photo': 789, 'Sydney': 743, 'Australia': 695, 'today': 243, 'people': 222, 'South': 173, 'would': 169, 'think': 167, 'Wales': 156}, 'Adelaide': {'Adelaide': 107, 'Australia': 70, 'South': 60, 'today': 54, 'people': 53, 'photo': 48, 'posted': 43, 'think': 36, 'would': 33, '@CheapFuelAdel': 33}, 'Melbourne': {'posted': 704, 'photo': 679, 'Melbourne': 409, 'Victoria': 341, 'Australia': 319, 'today': 260, 'people': 248, 'would': 202, 'great': 202, 'think': 187}, 'Newcas

In [ ]:
db10 = CloudantDB("topwordsmonth3")
db10.add_record(cdict,key="2021-04")


print(db10.curDB["2021-04"])
#print()

In [163]:
cdict=city_daily_popular_10words(startkey="2020-01")
cdict

Username: admin
Databases: ['topwordstotal', 'tweets_covid', 'tweets_keywords', 'tweets_no_keywords', 'tweets_raw', 'tweets_vaccine']
Accessing db: tweets_no_keywords


{'Adelaide': {'Adelaide': 281,
  'Australia': 203,
  'South': 163,
  'posted': 141,
  'photo': 134,
  'today': 125,
  'people': 82,
  'great': 80,
  'Happy': 68,
  'video': 68},
 'Ballarat': {'#walkies': 24,
  'Ballarat': 23,
  'Abstract': 16,
  '#zerowaste': 14,
  '#recycle': 13,
  '#MAFS': 12,
  '#abstractart': 12,
  '#artist': 11,
  '#recycling': 11,
  '#artwork': 11},
 'Brisbane': {'posted': 961,
  'photo': 938,
  'Brisbane': 489,
  'Queensland': 476,
  'Australia': 350,
  'today': 201,
  'people': 126,
  'great': 119,
  'night': 114,
  'first': 111},
 'Bunbury': {'Bunbury': 7,
  'Western': 4,
  'Australia': 4,
  'posted': 3,
  'photo': 3,
  '#housesitting': 2,
  'walking': 2,
  'today': 2,
  'Taking': 1,
  'break': 1},
 'Canberra': {'Canberra': 168,
  'posted': 101,
  'photo': 97,
  'Australian': 71,
  'Australia': 65,
  'great': 58,
  'eWasp': 58,
  'today': 55,
  'people': 52,
  'morning': 50},
 'Geelong': {'Geelong': 80,
  'Extreme': 44,
  'Board': 37,
  'Store': 25,
  'Victori

In [76]:
get_data_summary(db="no_keywords",viewType="zone")

Username: admin
Databases: ['test', 'topwords', 'tweets_covid', 'tweets_keywords', 'tweets_no_keywords', 'tweets_raw', 'tweets_vaccine']
Accessing db: tweets_no_keywords


{'Adelaide': 0.693848,
 'Ballarat': 0.715091,
 'Brisbane': 0.710385,
 'Bunbury': 0.80026,
 'Canberra': 0.690317,
 'Geelong': 0.678222,
 'Hobart': 0.690671,
 'Melbourne': 0.704503,
 'Newcastle': 0.75033,
 'Perth': 0.700816,
 'Sydney': 0.723348}

In [165]:
#from database_cloudant import CloudantDB
#from cloudant.design_document import DesignDocument
db10 = CloudantDB("topwordsmonth3")
db10.add_record(cdict,key="2021-04")


print(db10.curDB["2021-04"])
#print()
#print(db10.get_data())


Username: admin
Databases: ['topwordstotal', 'tweets_covid', 'tweets_keywords', 'tweets_no_keywords', 'tweets_raw', 'tweets_vaccine']
creating db: topwordsmonth3
Accessing db: topwordsmonth3
{'_id': '2021-04', 'Adelaide': {'Adelaide': 281, 'Australia': 203, 'South': 163, 'posted': 141, 'photo': 134, 'today': 125, 'people': 82, 'great': 80, 'Happy': 68, 'video': 68}, 'Ballarat': {'#walkies': 24, 'Ballarat': 23, 'Abstract': 16, '#zerowaste': 14, '#recycle': 13, '#MAFS': 12, '#abstractart': 12, '#artist': 11, '#recycling': 11, '#artwork': 11}, 'Brisbane': {'posted': 961, 'photo': 938, 'Brisbane': 489, 'Queensland': 476, 'Australia': 350, 'today': 201, 'people': 126, 'great': 119, 'night': 114, 'first': 111}, 'Bunbury': {'Bunbury': 7, 'Western': 4, 'Australia': 4, 'posted': 3, 'photo': 3, '#housesitting': 2, 'walking': 2, 'today': 2, 'Taking': 1, 'break': 1}, 'Canberra': {'Canberra': 168, 'posted': 101, 'photo': 97, 'Australian': 71, 'Australia': 65, 'great': 58, 'eWasp': 58, 'today': 55, 

In [164]:
cdict

{'Adelaide': {'Adelaide': 281,
  'Australia': 203,
  'South': 163,
  'posted': 141,
  'photo': 134,
  'today': 125,
  'people': 82,
  'great': 80,
  'Happy': 68,
  'video': 68},
 'Ballarat': {'#walkies': 24,
  'Ballarat': 23,
  'Abstract': 16,
  '#zerowaste': 14,
  '#recycle': 13,
  '#MAFS': 12,
  '#abstractart': 12,
  '#artist': 11,
  '#recycling': 11,
  '#artwork': 11},
 'Brisbane': {'posted': 961,
  'photo': 938,
  'Brisbane': 489,
  'Queensland': 476,
  'Australia': 350,
  'today': 201,
  'people': 126,
  'great': 119,
  'night': 114,
  'first': 111},
 'Bunbury': {'Bunbury': 7,
  'Western': 4,
  'Australia': 4,
  'posted': 3,
  'photo': 3,
  '#housesitting': 2,
  'walking': 2,
  'today': 2,
  'Taking': 1,
  'break': 1},
 'Canberra': {'Canberra': 168,
  'posted': 101,
  'photo': 97,
  'Australian': 71,
  'Australia': 65,
  'great': 58,
  'eWasp': 58,
  'today': 55,
  'people': 52,
  'morning': 50},
 'Geelong': {'Geelong': 80,
  'Extreme': 44,
  'Board': 37,
  'Store': 25,
  'Victori

In [159]:
db10 = CloudantDB("topwordstotal")
db10.add_record(cdict，key="2021-04")


print(db10.curDB)

print(db10.get_data())


SyntaxError: invalid character in identifier (<ipython-input-159-d1a65d093ba0>, line 2)

In [119]:
print(db10.curDB)

{}


In [ ]:
db10 = CloudantDB("topwordsmonth3")
db10.add_record(cdict,key="2021-04")


print(db10.curDB["2021-04"])
print()
print(db10.get_data())


In [187]:
# -*- coding: utf-8 -*-
"""
Created on Wed May 19 00:40:53 2021

@author: Windwalker
"""
USERNAME='admin'
PASSWORD = 'password'
URL = 'http://172.26.131.218:5984'
from cloudant.client import CouchDB
from cloudant.design_document import DesignDocument
from cloudant.view import View

class CloudantDB():
    def __init__(self,db_name,username = USERNAME,password=PASSWORD,url = URL,partition=True):
        self.client = CouchDB(USERNAME, PASSWORD, url=URL, connect=True)
        self.session = self.client.session()
        self.curDB = None
        print('Username: {0}'.format(self.session['userCtx']['name']))
        print('Databases: {0}'.format(self.client.all_dbs()))
        self.accessDB(db_name,partition=True)
    
    def accessDB(self,db_name,partition=True):
        if(db_name in self.client.all_dbs()):
            self.curDB = self.client[db_name]
        else:
            self.curDB = self.client.create_database(db_name)
            print("creating db:",db_name)
        if self.curDB.exists():
            print('Accessing db:',db_name)
    
    def add_record(self, json_record, db_name=None, key=None):
        if(key is not None and key not in self.curDB):
            json_record['_id'] = key
            self.curDB.create_document(json_record)
        elif('id' in json_record.keys()):
            hash_string = str(hash(json_record['id']))
            if(hash_string not in self.curDB):
                json_record['_id'] = hash_string
                record = json_record
                self.curDB.create_document(record)
            else:
                print("already exist")
    
    def delete_record(self,recordID):
        if(recordID in self.curDB):
            my_document = self.curDB[recordID]
            my_document.delete()
        elif(str(hash(recordID)) in self.curDB):
            my_document = self.curDB[str(hash(recordID))]
            my_document.delete()
        else:
            print("deletion err, record not found")
    def delete_all_records(self):
        for doc in self.curDB:
            doc.delete()
    
    def deleteDB(self,db_name):
        self.client.delete_database(db_name)
        self.curDB = None
        print(db_name,"removed")
    
    def get_data(self,n=10000):
        data = []
        count = 0
        for document in self.curDB:
            data.append(document)
            count+=1
            if(count>n):
                break
        return data
    
    def close(self):
        self.client.disconnect()
        
    def allDB(self):
        all_dbs = self.client.all_dbs()
        print('Databases: {0}'.format(all_dbs))
        return all_dbs
"""
reduce_func = "function(key, values, rereduce) {var result = 0;for(var i = 0; i < values.length; i++){value = parseFloat(values[i]);result+=value;}return result.toString();}"
db2 = CloudantDB("tweets_no_keywords")

ddoc = DesignDocument(db2.curDB, '_design/ddoc001')
ddoc.fetch()
ddoc.add_view("view_zone",'function(doc){emit(doc.zone,parseFloat(doc.sentiment_score))}',"_stats")
ddoc.save()

view = View(ddoc, 'view_zone')
# Assuming that 'view001' exists as part of the
# design document ddoc in the remote database...
# Use view as a callable
for row in view(limit=100,group=True)['rows']:
    print(row)
for view_name, view in ddoc.iterviews():
    print("------",view_name,view)

db2 = CloudantDB("tweets_no_keywords")
ddoc = DesignDocument(db2.curDB,'_design/ddoc001')
ddoc.fetch()
ddoc.add_view("view_time","function(doc){emit(doc.created_at.split(' ')[0], parseFloat(doc.sentiment_score))}","_stats")
ddoc.save()
view = View(ddoc, 'view_time')
for row in view(limit=100,reduce=True,group=True)['rows']:
    print(row)
"""

'\nreduce_func = "function(key, values, rereduce) {var result = 0;for(var i = 0; i < values.length; i++){value = parseFloat(values[i]);result+=value;}return result.toString();}"\ndb2 = CloudantDB("tweets_no_keywords")\n\nddoc = DesignDocument(db2.curDB, \'_design/ddoc001\')\nddoc.fetch()\nddoc.add_view("view_zone",\'function(doc){emit(doc.zone,parseFloat(doc.sentiment_score))}\',"_stats")\nddoc.save()\n\nview = View(ddoc, \'view_zone\')\n# Assuming that \'view001\' exists as part of the\n# design document ddoc in the remote database...\n# Use view as a callable\nfor row in view(limit=100,group=True)[\'rows\']:\n    print(row)\nfor view_name, view in ddoc.iterviews():\n    print("------",view_name,view)\n\ndb2 = CloudantDB("tweets_no_keywords")\nddoc = DesignDocument(db2.curDB,\'_design/ddoc001\')\nddoc.fetch()\nddoc.add_view("view_time","function(doc){emit(doc.created_at.split(\' \')[0], parseFloat(doc.sentiment_score))}","_stats")\nddoc.save()\nview = View(ddoc, \'view_time\')\nfor ro

In [182]:

def add_months(month):
    start=month+"-01 00:00:00"
    newyear=False
    if month[-2:]=="12":
        endmonth="01"
        newyear=True
    elif month[-2:]=="11":
        endmonth="12"
    elif month[-2:]=="10":
        endmonth="11"
    elif month[-2:]=="09":
        endmonth="10"
    else:
        endmonth="0"+str(1+int(month[-1]))
    end=month[:-2]+endmonth+"-01 00:00:00"
    if newyear:
        end=month[:-4]+str(1+int(month[3]))+"-"+endmonth+"-01 00:00:00"
    return start,end  

def city_daily_popular_10words(tweetlist, startmonth='2021-01' ):
    (start,end)=add_months(startmonth)
    city=["Sydney","Newcastle","Canberra", "Melbourne","Ballarat","Geelong","Hobart","Perth","Brisbane","Adelaide"]
    alltext={}
    month=[]
    startlist=[]
    endlist=[]
    for m in range(12):
        month.append(start[:7])
        startlist.append(start)
        endlist.append(end)
        (start,end)=add_months(end[:7])
    for m in month:
        toplist={}
        for c in city:
            toplist[c]=[]  
        
        alltext[m]=toplist
    for k in tweetlist:
        if k['created_at']<=end and k['created_at']>start: 
            for c in city:
                if k["zone"] in c:
                    alltext[c].append(k["text"])
    popdict={}  
    for m in month:
        worddict={}
        for c in city:
            worddict[c]=top_words(alltext[m][c])
        popdict[m]=worddict
    return popdict